In [65]:
from langchain.memory import ConversationSummaryBufferMemory # 메모리에 보내온 메세지의 수를 저장한다. Combine Buffer Memory & Summary Memory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=.1)

# 대화 중의 엔티티의 Knowledge Graph를 만듭니다.
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=160,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{question}"),
])

def load_memory():
    return memory.load_memory_variables({})["history"]

chain = prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
        "history": load_memory(),
    })
    memory.save_context({"input": question}, {"output": result.content},)
    print(result)

In [66]:
invoke_chain("My name is Marco")

content='Hello Marco! How can I assist you today?'


In [67]:
invoke_chain("What is my Name?")

content='Your name is Marco. How can I assist you today, Marco?'
